In [1]:
import pandas as pd
import numpy as np
import pickle

### Data

Izquierda Diario

In [2]:
df_izq = pickle.load(open('data/df_clean_izq.pkl','rb'))

In [3]:
df_inflation_izq = df_izq[df_izq.topics == 'inflation']

In [4]:
df_exchange_izq = df_izq[df_izq.topics == 'exchange']

In [5]:
corpus_izq = df_inflation_izq.append(df_exchange_izq)

Derecha Diario

In [6]:
df_der = pickle.load(open('data/df_clean_der.pkl','rb'))

In [7]:
df_inflation_der = df_der[df_der.topics == 'inflation']

In [8]:
df_exchange_der = df_der[df_der.topics == 'exchange']

In [9]:
corpus_der = df_inflation_der.append(df_exchange_der)

### Tokenization

In [10]:
import re
import unidecode

In [11]:
from nltk.corpus import stopwords
import spacy
from spacy.lang.es import Spanish

In [12]:
parser = Spanish()
stopwords_sp = stopwords.words('spanish') + ['si','dia', 'vez', 'fin','dias', 'dos', 'mas','ano', 'asi','puede', 'mes','incluso']

In [13]:
def tokenizer(texto):

    alphanumeric = re.sub(r'([^\s\w]|_)+', '', texto).lower()
    no_accents = unidecode.unidecode(alphanumeric)
    
    tokens = parser(no_accents)
    
    tokens_list = [str(token) for token in tokens]
    
    tokens_clean = [token for token in tokens_list if token not in stopwords_sp and token.isalpha()]

    return tokens_clean

In [14]:
tokens_izq = [tokenizer(text) for text in corpus_izq.body]

In [15]:
tokens_der = [tokenizer(text) for text in corpus_der.body]

### Gensim

Give ids to tokens with gensim Dictionary

In [16]:
# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [17]:
from gensim.corpora import Dictionary

In [18]:
# tokens input
dictionary_izq = Dictionary(tokens_izq)
dictionary_izq.filter_extremes(no_below=2, no_above=0.97, keep_n=None)

# convert each document into the gensim bag-of-words
corpus_izq = [dictionary_izq.doc2bow(text) for text in tokens_izq]

In [19]:
# tokens input
dictionary_der = Dictionary(tokens_der)
dictionary_der.filter_extremes(no_below=2, no_above=0.97, keep_n=None)

# convert each document into the gensim bag-of-words
corpus_der = [dictionary_der.doc2bow(text) for text in tokens_der]

### Latent Dirichlet Allocation (LDA)
Check the unsupervised word classification using LDA

In [20]:
from gensim import matutils, models
import scipy.sparse

Izquierda Diario

In [21]:
# make a dictionary with the ids of each word assigned by Gensim dictionary method

id_word_izq = dict((v, k) for k, v in dictionary_izq.token2id.items())

In [22]:
# we already know that there are two topics

lda_izq = models.LdaModel(corpus=corpus_izq, id2word=id_word_izq, num_topics=2, passes=20, random_state= 30)

In [23]:
lda_izq.print_topics()

[(0,
  '0.034*"dolar" + 0.012*"dolares" + 0.011*"millones" + 0.010*"banco" + 0.010*"us" + 0.009*"central" + 0.008*"deuda" + 0.008*"guzman" + 0.008*"oficial" + 0.008*"gobierno"'),
 (1,
  '0.020*"precios" + 0.014*"inflacion" + 0.013*"aumento" + 0.012*"suba" + 0.011*"canasta" + 0.010*"alimentos" + 0.010*"productos" + 0.009*"salarios" + 0.009*"indec" + 0.008*"trabajadores"')]

In [24]:
import pyLDAvis.gensim

lda_display = pyLDAvis.gensim.prepare(lda_izq, corpus_izq, dictionary_izq, sort_topics=False)

pyLDAvis.display(lda_display)

Derecha Diario

In [25]:
# make a dictionary with the ids of each word assigned by Gensim dictionary method

id_word_der = dict((v, k) for k, v in dictionary_der.token2id.items())

/home/guido/anaconda3/envs/dh/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
# we already know that there are two topics

lda_der = models.LdaModel(corpus=corpus_der, id2word=id_word_der, num_topics=2, passes=20, random_state= 30)

/home/guido/anaconda3/envs/dh/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
lda_der.print_topics()

/home/guido/anaconda3/envs/dh/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.011*"inflacion" + 0.009*"precios" + 0.008*"dolar" + 0.008*"gobierno" + 0.008*"mercado" + 0.007*"economia" + 0.007*"tipo" + 0.006*"aumento" + 0.005*"monetaria" + 0.004*"argentina"'),
 (1,
  '0.008*"precios" + 0.008*"gobierno" + 0.008*"inflacion" + 0.007*"dolar" + 0.006*"mercado" + 0.006*"monetaria" + 0.006*"economia" + 0.005*"argentina" + 0.005*"banco" + 0.005*"pais"')]

In [28]:
lda_display = pyLDAvis.gensim.prepare(lda_der, corpus_der, dictionary_der, sort_topics=False)
pyLDAvis.display(lda_display)

/home/guido/anaconda3/envs/dh/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Filtering by tags

In [29]:
import nltk

/home/guido/anaconda3/envs/dh/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
# tag is not accurate
nltk.pos_tag(tokens_izq[0][:10])

/home/guido/anaconda3/envs/dh/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[('inflacion', 'NN'),
 ('detiene', 'NN'),
 ('mayoria', 'NN'),
 ('familias', 'NN'),
 ('presenta', 'NN'),
 ('dificil', 'NN'),
 ('afrontar', 'NN'),
 ('indice', 'NN'),
 ('precios', 'NNS'),
 ('mayoristas', 'NNS')]

In [31]:
from polyglot.text import Text
!polyglot download embeddings2.es pos2.es

/home/guido/anaconda3/envs/dh/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[polyglot_data] Downloading package embeddings2.es to
[polyglot_data]     /home/guido/polyglot_data...
[polyglot_data]   Package embeddings2.es is already up-to-date!
[polyglot_data] Downloading package pos2.es to
[polyglot_data]     /home/guido/polyglot_data...
[polyglot_data]   Package pos2.es is already up-to-date!


In [32]:
def noun_only(tokens):
    nouns = []
    for token in tokens:
        word_t = Text(token, hint_language_code='es')
        if word_t.pos_tags[0][1] == 'NOUN':
            nouns.append(word_t.pos_tags[0][0])
    return nouns

/home/guido/anaconda3/envs/dh/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
tokens_nouns_izq = [noun_only(token) for token in tokens_izq]
tokens_nouns_der = [noun_only(token) for token in tokens_der]

/home/guido/anaconda3/envs/dh/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### New Noun Dictionary + LDA

In [34]:
# tokens input
dictionary_izq = Dictionary(tokens_nouns_izq)
dictionary_izq.filter_extremes(no_below=2, no_above=0.97, keep_n=None)

# convert each document into the gensim bag-of-words
corpus_izq = [dictionary_izq.doc2bow(text) for text in tokens_nouns_izq]

/home/guido/anaconda3/envs/dh/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [35]:
# tokens input
dictionary_der = Dictionary(tokens_nouns_der)
dictionary_der.filter_extremes(no_below=2, no_above=0.97, keep_n=None)

# convert each document into the gensim bag-of-words
corpus_der = [dictionary_der.doc2bow(text) for text in tokens_nouns_der]

/home/guido/anaconda3/envs/dh/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Izquierda Diario

In [36]:
id_word_izq = dict((v, k) for k, v in dictionary_izq.token2id.items())

/home/guido/anaconda3/envs/dh/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [37]:
lda_izq = models.LdaModel(corpus=corpus_izq, id2word=id_word_izq, num_topics=2, passes=20, random_state= 30)

/home/guido/anaconda3/envs/dh/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [38]:
lda_izq.print_topics()

/home/guido/anaconda3/envs/dh/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.038*"banco" + 0.032*"deuda" + 0.030*"gobierno" + 0.029*"bonos" + 0.025*"economia" + 0.023*"medidas" + 0.020*"brecha" + 0.018*"mercado" + 0.018*"tipo" + 0.015*"presion"'),
 (1,
  '0.064*"precios" + 0.030*"alimentos" + 0.028*"productos" + 0.028*"salarios" + 0.023*"incremento" + 0.021*"aumentos" + 0.018*"gobierno" + 0.017*"bebidas" + 0.017*"meses" + 0.016*"ingresos"')]

In [39]:
lda_display = pyLDAvis.gensim.prepare(lda_izq, corpus_izq, dictionary_izq, sort_topics=False)
pyLDAvis.display(lda_display)

/home/guido/anaconda3/envs/dh/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Derecha Diario

In [40]:
id_word_der = dict((v, k) for k, v in dictionary_der.token2id.items())

/home/guido/anaconda3/envs/dh/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [41]:
lda_der = models.LdaModel(corpus=corpus_der, id2word=id_word_der, num_topics=2, passes=20, random_state= 30)

/home/guido/anaconda3/envs/dh/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [42]:
lda_der.print_topics()

/home/guido/anaconda3/envs/dh/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.037*"mercado" + 0.029*"tipo" + 0.029*"gobierno" + 0.025*"banco" + 0.018*"medidas" + 0.016*"brecha" + 0.016*"pais" + 0.014*"demanda" + 0.013*"economia" + 0.011*"divisas"'),
 (1,
  '0.047*"precios" + 0.027*"economia" + 0.024*"gobierno" + 0.018*"nivel" + 0.014*"dinero" + 0.014*"pais" + 0.012*"indice" + 0.011*"actividad" + 0.011*"medidas" + 0.011*"dato"')]

In [43]:
lda_display = pyLDAvis.gensim.prepare(lda_der, corpus_der, dictionary_der, sort_topics=False)
pyLDAvis.display(lda_display)

/home/guido/anaconda3/envs/dh/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
